# Data Science project: CIRI

## Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tqdm
import os

from enum import Enum
from torch.utils.data import DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

## Parameters

## Generic setup

Set the device. Using CUDA on CUDA-enabled devices speeds up the use of convolutional networks significantly.

In [2]:
cuda_enabled = torch.cuda.is_available()
device = torch.device("cuda:1" if cuda_enabled else "cpu")

We will be using a random number generator every now and then throughout this notebook. By initializing it here, we can change a single seed to obtain the same/different results.

In [3]:
rng = np.random.default_rng(0)

We create the descriptors through transfer learning. We remove the last fully-connected (classification) layers of pre-trained models and use the output of the convolutional part of the respective model as descriptors. We do this for:

- VGG16

These models were chosen because of ... **TODO: List reasons for choosing models**

## Load datasets

Because the images were pre-processed in another notebook, they must now be loaded into this one. To do so, we use a custom class taken from https://medium.com/codex/saving-and-loading-transformed-image-tensors-in-pytorch-f37b4daa9658.

In [4]:
class prepped_data(torch.utils.data.Dataset):
    def __init__(self, img, mask):
        self.img = img  #img path
        self.mask = mask  #mask path
        self.len = len(os.listdir(self.img))

    def __getitem__(self, index):
        ls_img = sorted(os.listdir(self.img))
        ls_mask = sorted(os.listdir(self.mask))

        img_file_path = os.path.join(self.img, ls_img[index])
        img_tensor = torch.load(img_file_path)

        mask_file_path = os.path.join(self.mask, ls_mask[index])
        mask_tensor = torch.load(mask_file_path)

        return img_tensor, mask_tensor

    def __len__(self):
        return self.len

In [5]:
data_train = prepped_data('./content/train_loader/img', './content/train_loader/target')
data_test = prepped_data('./content/test_loader/img', './content/test_loader/target')

with open('./content/data_classes.bin', 'rb') as file:
    data_classes = pickle.load(file)

In [6]:
class DataType(Enum):
    TRAIN = "train"
    TEST = "test"


class Dataset:
    """
    Represents a dataset from which data, targets and dataloaders can be easily
    extracted. Used mostly as a Data Transfer Object to reduce the need to pass
    high numbers of parameters.
    """

    def __init__(self, data, classes):
        """
        Keyword arguments:
        data   -- dictionary of dataset labels mapped to corresponding Datasets
        classes -- a list of all classes that exist in the Datasets
        """
        self.data = data
        self.all_classes = classes

        self.targets = {}
        for label, data in self.data.items():
            self.targets[label] = self._get_targets(data)

        self.dataloaders = {}
        for label, data in self.data.items():
            self.dataloaders[label] = DataLoader(
                data,
                batch_size=4,
                shuffle=False
            )

    def _get_targets(self, data):
        return [target for (_, target) in data]


dataset = Dataset(
    {
        DataType.TRAIN: data_train,
        DataType.TEST: data_test
    },
    data_classes
)

## Initialize models

In [7]:
class Model():
    """
    Represents a generic convolutional model that is used for transfer
    learning, from which descriptors can be extracted for classification.
    """

    def __init__(self, model, name):
        self.model = model
        self.name = name

        for param in self.model.parameters():
            # As the model will not be trained, gradients are not required.
            # Disabling them speeds up performance.
            param.requires_grad = False

        self.model.eval()
        self.remove_classifier()

    def remove_classifier(self):
        """
        Removes the last fully connected layer (the classifier) from the
        model, allowing the extraction of descriptors.
        """
        raise NotImplementedError()

    def to(self, device):
        self.model = self.model.to(device)

    def descriptors(self, dataloader, as_numpy=True):
        """
        Returns the output of the model for all items in the dataloader.
        """
        outputs = []
        with torch.no_grad():
            for data, targets in tqdm.tqdm(dataloader):
                data = data.to(device)
                output = self.model(data).detach()
                outputs.extend(output)

        if as_numpy:
            outputs = np.array([output.cpu().numpy() for output in outputs])

        return outputs


models = []
if cuda_enabled:
    torch.cuda.empty_cache()

### VGG16

In [8]:
class Vgg16Model(Model):
    def __init__(self):
        model = torchvision.models.vgg16(weights="DEFAULT")
        super().__init__(model, "VGG16")

    def remove_classifier(self):
        self.model.classifier = nn.Identity()


vgg16 = Vgg16Model()
vgg16.to(device)

models.append(vgg16)

### Inception V3

In [9]:
class InceptionV3Model(Model):
    def __init__(self):
        model = torchvision.models.inception_v3(weights="DEFAULT")
        super().__init__(model, "InceptionV3")

    def remove_classifier(self):
        self.model.fc = nn.Identity()


inception = InceptionV3Model()
inception.to(device)

models.append(inception)

## Baseline

### Obtain image descriptors

In [10]:
class DescriptorManager:
    """
    Handles the loading, saving and extraction of data descriptors.
    """

    def __init__(self, model, base_path="./data/descriptors"):
        self.model = model
        self.base_path = base_path
        self.descriptors = {}

    def _create_path(self, label):
        """
        Creates a standardised path for descriptors with the provided model name
        and label to prevent inconsistencies between loading and saving descriptors.
        """
        return f"{self.base_path}/{self.model.name}/descriptors_{label}.bin"

    def save_descriptors(self, descriptors, label):
        """
        Saves the provided descriptor to the file system at a path derived from
        the model name and label.
        """
        path = self.create_path(label)
        os.makedirs(os.path.dirname(path), exist_ok=True)

        with open(path, "wb") as file:
            pickle.dump(descriptors, file)

    def extract_descriptors(self, dataloader, label):
        """
        Extracts descriptors from the provided model and saves them to this
        DescriptorManager.
        """
        descriptors = self.model.descriptors(dataloader, True)
        self.descriptors[label] = descriptors

        return descriptors

    def load_descriptors(self, label):
        """
        Loads and returns the provided descriptors from the file system, based
        on the provided model name and label.
        """
        path = self._create_path(label)

        with open(path, "rb") as file:
            self.descriptors[label] = pickle.load(file)

            return self.descriptors[label]

    def get_descriptors(self, dataloader, label):
        """
        Returns cached descriptors, returns descriptors from the file system or
        extracts and saves new descriptors from the model, based on which is
        available.

        Keyword arguments:
        dataloader -- DataLoader with data for which descriptors are retrieved
        label -- the label for this data (e.g. "train", "test", "fold1", etc.)
        """
        if label in self.descriptors:
            return self.descriptors[label]

        try:
            return self.load_descriptors(label)
        except FileNotFoundError:
            descriptors = self.extract_descriptors(dataloader, label)
            self.save_descriptors(descriptors, label)

            return descriptors

### Perform k-nearest neighbors (kNN) classification

First, we define the number of neighbors using which we would like to perform classification.

In [11]:
n_neighbors_list = [2, 3, 5, 8, 13, 18, 21, 34]

For each `n_neighbors` in the defined list, we perform classification and calculate the accuracy.

In [12]:
class DescriptorAssessor:
    def __init__(self, descriptor_manager):
        self.descriptor_manager = descriptor_manager

    def _print_accuracies(self, accuracies, label):
        for n_neighbors, accuracy in accuracies.items():
            print(f"{self.model.name} accuracy on {label} set for {n_neighbors}-neighbor classification: {accuracy}")

    def _get_descriptors(self, dataset, data_types):
        descriptors = {}

        for data_type in data_types:
            descriptors[data_type] = self.descriptor_manager.get_descriptors(
                dataset.dataloaders[data_type],
                data_type
            )

        return descriptors

    def assess(self, dataset, n_neighbors_list, train_type, assess_types):
        """
        Assess the performance of kNN-classifiers on model descriptors for
        different numbers of neighbors.

        Keyword arguments:
        dataset -- dictionary of data types (train, test, ...) mapped to Subsets
        n_neighbors_list -- list of numbers of neighbors to assess
        train_type -- the data type to which the classifier should be fit
        assess_types -- the types on which to assess the classifier accuracy
        """
        descriptors = self._get_descriptors(
            dataset,
            assess_types
        )

        predictions = dict([(assess_type, {}) for assess_type in assess_types])
        accuracies = dict([(assess_type, {}) for assess_type in assess_types])
        precisions = dict([(assess_type, {}) for assess_type in assess_types])
        recalls = dict([(assess_type, {}) for assess_type in assess_types])
        f1_scores = dict([(assess_type, {}) for assess_type in assess_types])

        for n_neighbors in n_neighbors_list:
            classifier = KNeighborsClassifier(n_neighbors=n_neighbors).fit(
                descriptors[train_type],
                dataset.targets[train_type]
            )

            for assess_type in assess_types:
                predictions[assess_type][n_neighbors] = classifier.predict(
                    descriptors[assess_type]
                )
                accuracies[assess_type][n_neighbors] = accuracy_score(
                    y_true=dataset.targets[assess_type],
                    y_pred=predictions[assess_type][n_neighbors]
                )
                precisions[assess_type][n_neighbors] = precision_score(
                    y_true=dataset.targets[assess_type],
                    y_pred=predictions[assess_type][n_neighbors],
                    average="weighted"
                )
                recalls[assess_type][n_neighbors] = recall_score(
                    y_true=dataset.targets[assess_type],
                    y_pred=predictions[assess_type][n_neighbors],
                    average="weighted"
                )
                f1_scores[assess_type][n_neighbors] = f1_score(
                    y_true=dataset.targets[assess_type],
                    y_pred=predictions[assess_type][n_neighbors],
                    average="weighted"
                )

        return (descriptors, predictions, accuracies, precisions, recalls, f1_scores)

In [13]:
class Visualiser:
    def __init__(self, model):
        self.model = model

    def _print_accuracies(self, accuracies, label):
        for n_neighbors, accuracy in accuracies.items():
            print(f"{self.model.name} accuracy on {label} set for {n_neighbors}-neighbor classification: {accuracy}")

    def _plot_metric(self, ax, metric, label):
        return ax.plot(
            metric.keys(),
            list(metric.values()),
            label=label,
            marker="o"
        )

    def visualise_metrics_per_n(self, accuracies, precisions, recalls, f1_scores):
        fig, ax = plt.subplots()

        self._print_accuracies(accuracies[DataType.TEST], DataType.TEST)
        print(accuracies[DataType.TEST])
        self._plot_metric(ax, accuracies[DataType.TEST], "accuracy")
        self._plot_metric(ax, precisions[DataType.TEST], "precision")
        self._plot_metric(ax, recalls[DataType.TEST], "recall")
        self._plot_metric(ax, f1_scores[DataType.TEST], "f1_score")

        ax.legend()
        ax.set_title(f"kNN-classification performance using {self.model.name} descriptors")
        ax.set_xlabel("k (number of neighbours)")
        ax.set_ylabel("score")
        plt.show()

In [ ]:
for model in models:
    descriptor_manager = DescriptorManager(model)
    descriptor_assessor = DescriptorAssessor(descriptor_manager)

    descriptors, predictions, accuracies, precisions, recalls, f1_scores = descriptor_assessor.assess(
        dataset,
        n_neighbors_list,
        DataType.TRAIN,
        [DataType.TRAIN, DataType.TEST]
    )

    visualiser = Visualiser(model)
    visualiser.visualise_metrics_per_n(
        accuracies,
        precisions,
        recalls,
        f1_scores
    )

    best_n_test = max(
        accuracies[DataType.TEST],
        key=accuracies[DataType.TEST].get
    )

    cm = confusion_matrix(
        dataset.targets[DataType.TEST],
        predictions[DataType.TEST][best_n_test]
    )
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

/home/jovyan/2022-1B-DS-CIRI-Project/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/2022-1B-DS-CIRI-Project/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/2022-1B-DS-CIRI-Project/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/

### Visually assess predicted vs. target through PCA

In [ ]:
def plot_predictions_against_targets(fit_pca,
                                     predictions,
                                     targets,
                                     n_classes,
                                     cmap=None):
    """Draws a scatter plot of two-dimensional data which highlights
    differences between targets and predictions through distinct edge and fill
    colors."""

    if cmap is None:
        cmap = plt.get_cmap("rainbow")

    if isinstance(targets, list):
        targets = np.array(targets)

    colors_target = cmap(targets / n_classes)
    colors_predictions = cmap(predictions / n_classes)

    fig, ax = plt.subplots()
    # Plot targets (ground truth) with fill color representing target class
    ax.scatter(fit_pca[:, 0],
               fit_pca[:, 1],
               facecolors=colors_target,
               label="target")

    # Plot predictions with edge color representing predicted class
    ax.scatter(fit_pca[:, 0],
               fit_pca[:, 1],
               facecolors="none",
               edgecolors=colors_predictions,
               label="predicted")

    legend = ax.legend()
    # Set colors to black to indicate the legend is about marker type, rather
    # than color
    legend.legendHandles[0].set_color("black")
    legend.legendHandles[1].set_edgecolor("black")

    return (fig, ax)